In [0]:
%pip install gspread==5.12.4 

%restart_python

In [0]:
%load_ext autoreload
%autoreload 2 

In [0]:
import pandas as pd
import json
import gspread

In [0]:
def get_gspread_client_from_secret(scope='krista-ds-loc-scope',
                                   key = 'crafty_json_key'):
    
    try:
        crafty_json = dbutils.secrets.get(scope = 'krista-ds-loc-scope', key = 'crafty_json_key')
    except:
        print("Cannot get secret from scope for gspread!!")

    gc = gspread.service_account_from_dict(json.loads(crafty_json))

    print("Getting gspread client...")
    return gc

In [0]:
gc = get_gspread_client_from_secret()

def get_processing_rows_from_sheet_aso_monthly():

    #WORKBOOK_ID = "1WnCBGk1V1nEUeU7aPWBqrQXczmaMop6AbGmkDKkAQcE"
    ASO_MONTHLY_SS_URL = "https://docs.google.com/spreadsheets/d/1WnCBGk1V1nEUeU7aPWBqrQXczmaMop6AbGmkDKkAQcE/edit?gid=1033119277#gid=1033119277"

    WKSHT_NAME = "Validation and Url Tracking"

    source_ss = gc.open_by_url(ASO_MONTHLY_SS_URL)

    source_wksht = source_ss.worksheet(WKSHT_NAME)
    vals = source_wksht.get_all_values()

    headers, data = vals[0],vals[1:]
    df = pd.DataFrame(data, columns=headers)

    to_process_df = df[df.JobStatus == 'NotStarted']


    return to_process_df


def get_row_dicts(to_process_df:pd.DataFrame, 
                  source_cd:str )->list:

    list_slugs = []
    for i,row in to_process_df.iterrows():
        row_dict = row.to_dict()
        row_dict['source_cd'] = source_cd
        json_job_slug = json.dumps(row_dict)
        row['JsonJobSlug'] = json_job_slug
        list_slugs.append(row['JsonJobSlug'])

    return list_slugs

In [0]:
SOURCE_CD =  'ASO Monthly'

to_process_df = get_processing_rows_from_sheet_aso_monthly()

row_slugs= pd.DataFrame(get_row_dicts(to_process_df, SOURCE_CD),columns = ['JsonJobSlug'])

OUT_JSON = json.dumps(row_slugs.to_dict('records'))


In [0]:
#TODO: Process Languages 

In [0]:
dbutils.notebook.exit(OUT_JSON)